In [ ]:
Chapter 7. Purely functional parallelism

In [ ]:
7.1. Choosing data type and functions
7.1.1. A data type for parallel computations
7.1.2. Combining parallel computations
7.1.3. Explicit forking
7.2. Picking a representation
7.3. Refining the API
7.4. The algebra of an API
7.4.1. The law of mapping
7.4.2. The law of forking
